# CGAN Approach

inspired by 
https://medium.com/the-research-nest/exploring-gans-to-generate-synthetic-data-ca48f8a4b518

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

df_train = pd.read_csv("../data/processed/sorted_train_data_mm.csv")
df_val = pd.read_csv("../data/processed/sorted_val_data_mm.csv")

real_data = df_train.drop(columns="Class")
validation_data = df_val.drop(columns="Class")

real_labels = df_train.Class
validation_lables = df_val.Class

# One hot encode labels
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_labels = one_hot_encoder.fit_transform(np.array(real_labels).reshape(-1, 1))
one_hot_validation_labels = one_hot_encoder.fit_transform(
    np.array(validation_lables).reshape(-1, 1)
)

c:\Users\Max_G\anaconda3\envs\var_auto\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Max_G\anaconda3\envs\var_auto\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
# Constants
NOISE_DIM = 100
NUM_CLASSES = 2
NUM_FEATURES = 30
BATCH_SIZE = 32
TRAINING_STEPS = "EPCOH"

In [3]:
# Generator
def create_generator():
    noise_input = Input(shape=(NOISE_DIM,))
    class_input = Input(shape=(NUM_CLASSES,))
    merged_input = Concatenate()([noise_input, class_input])
    hidden = Dense(128, activation="relu")(merged_input)
    output = Dense(NUM_FEATURES, activation="linear")(hidden)
    model = Model(inputs=[noise_input, class_input], outputs=output)
    return model


# Discriminator
def create_discriminator():
    data_input = Input(shape=(NUM_FEATURES,))
    class_input = Input(shape=(NUM_CLASSES,))
    merged_input = Concatenate()([data_input, class_input])
    hidden = Dense(128, activation="relu")(merged_input)
    output = Dense(1, activation="sigmoid")(hidden)
    model = Model(inputs=[data_input, class_input], outputs=output)
    return model


# cGAN
def create_cgan(generator, discriminator):
    noise_input = Input(shape=(NOISE_DIM,))
    class_input = Input(shape=(NUM_CLASSES,))
    generated_data = generator([noise_input, class_input])
    validity = discriminator([generated_data, class_input])
    model = Model(inputs=[noise_input, class_input], outputs=validity)
    return model


# Create and compile the Discriminator
discriminator = create_discriminator()
discriminator.compile(loss="binary_crossentropy", optimizer=Adam())

# Create the Generator
generator = create_generator()

# Create the GAN
gan = create_cgan(generator, discriminator)

# Ensure that only the generator is trained
discriminator.trainable = False

gan.compile(loss="binary_crossentropy", optimizer=Adam())



In [4]:
def evaluate_on_validation(generator, discriminator, validation_data, validation_labels, BATCH_SIZE, NOISE_DIM):
    # Calculate the number of batches
    num_batches = len(validation_data) // BATCH_SIZE

    # Initialize loss accumulators
    discriminator_losses = []
    generator_losses = []

    # Iterate over validation data in batches
    for i in range(num_batches):
        # Calculate start and end indices for the current batch
        start_idx = i * BATCH_SIZE
        end_idx = (i + 1) * BATCH_SIZE

        # Select a batch of validation data
        real_batch = validation_data[start_idx:end_idx]
        labels_batch = validation_labels[start_idx:end_idx]

        # Generate noise for the generator
        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))

        # Generate fake data using the generator
        generated_batch = generator.predict([noise, labels_batch])

        # Evaluate discriminator loss on the validation batch
        real_loss = discriminator.test_on_batch([real_batch, labels_batch], np.ones((BATCH_SIZE, 1)))
        fake_loss = discriminator.test_on_batch([generated_batch, labels_batch], np.zeros((BATCH_SIZE, 1)))
        discriminator_losses.append(0.5 * np.add(real_loss, fake_loss))

        # Evaluate generator loss on the validation batch
        generator_losses.append(gan.test_on_batch([noise, labels_batch], np.ones((BATCH_SIZE, 1))))

    # Calculate average losses over all batches
    avg_discriminator_loss = np.mean(discriminator_losses)
    avg_generator_loss = np.mean(generator_losses)

    return avg_discriminator_loss, avg_generator_loss


def train_gan(
    generator,
    discriminator,
    gan,
    real_data,
    one_hot_labels,
    validation_data,
    validation_labels,
    NUM_EPOCHS,
    BATCH_SIZE,
    NOISE_DIM,
):
    # Determine the number of steps per epoch
    steps_per_epoch = len(real_data) // BATCH_SIZE
    print(f"There are {steps_per_epoch} steps per epoch")
    # Train GAN for the specified number of epochs
    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")

        # Iterate over the entire dataset sequentially
        for step in range(steps_per_epoch):
            print(step)

            # Calculate the start and end indices for the current batch
            start_idx = step * BATCH_SIZE
            end_idx = (step + 1) * BATCH_SIZE

            # Select a batch of real data with labels
            real_batch = real_data.iloc[start_idx:end_idx].values
            labels_batch = one_hot_labels[start_idx:end_idx]

            # Generate a batch of new data
            noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
            generated_batch = generator.predict([noise, labels_batch])

            # Train the discriminator
            real_loss = discriminator.train_on_batch(
                [real_batch, labels_batch], np.ones((BATCH_SIZE, 1))
            )
            fake_loss = discriminator.train_on_batch(
                [generated_batch, labels_batch], np.zeros((BATCH_SIZE, 1))
            )
            discriminator_loss = 0.5 * np.add(real_loss, fake_loss)

            # Train the generator
            generator_loss = gan.train_on_batch(
                [noise, labels_batch], np.ones((BATCH_SIZE, 1))
            )
        # Evaluate models on validation set
        validation_loss = evaluate_on_validation(
            generator,
            discriminator,
            validation_data,
            validation_labels,
            BATCH_SIZE,
            NOISE_DIM,
        )

        # Print loss at the end of each epoch
        print(
            f"Epoch {epoch + 1}/{NUM_EPOCHS}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}, Validation Loss: {validation_loss}"
        )


# Example usage
train_gan(
    generator,
    discriminator,
    gan,
    real_data,
    one_hot_labels,
    validation_data,
    one_hot_validation_labels,
    2,
    BATCH_SIZE,
    NOISE_DIM,
)

There are 6230 steps per epoch
Epoch 1/2
0
1/1 [==============================] - 0s 123ms/step
1
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 22ms/step
3
1/1 [==============================] - 0s 22ms/step
4
1/1 [==============================] - 0s 21ms/step
5
1/1 [==============================] - 0s 21ms/step
6
1/1 [==============================] - 0s 22ms/step
7
1/1 [==============================] - 0s 21ms/step
8
1/1 [==============================] - 0s 21ms/step
9
1/1 [==============================] - 0s 25ms/step
10
1/1 [==============================] - 0s 23ms/step
11
1/1 [==============================] - 0s 22ms/step
12
1/1 [==============================] - 0s 21ms/step
13
1/1 [==============================] - 0s 23ms/step
14
1/1 [==============================] - 0s 24ms/step
15
1/1 [==============================] - 0s 25ms/step
16
1/1 [==============================] - 0s 23ms/step
17
1/1 [=========================

In [5]:
path = f"../models/CGAN/WEIGHTS_{TRAINING_STEPS}/"

In [5]:
path = f"../models/CGAN/WEIGHTS_{TRAINING_STEPS}/"

gan.save_weights(path + "gan_weights")
generator.save_weights(path + "generator_weights")
discriminator.save_weights(path + "disc_weights")

In [6]:
# Load weights for GAN
gan.load_weights(path + "gan_weights")
# Load weights for generator
generator.load_weights(path + "generator_weights")
# Load weights for discriminator
discriminator.load_weights(path + "disc_weights")

In [7]:
# Generate instances for a given class
def generate_data(generator, data_class, num_instances):
    one_hot_class = one_hot_encoder.transform(np.array([[data_class]]))
    noise = np.random.normal(0, 1, (num_instances, NOISE_DIM))
    generated_data = generator.predict(
        [noise, np.repeat(one_hot_class, num_instances, axis=0)]
    )
    return pd.DataFrame(generated_data, columns=[df_train.columns[:-1]])

In [8]:
# Generate 50 instances for each class
synthetic_data_class_0 = generate_data(generator, 0, 56870)
synthetic_data_class_1 = generate_data(generator, 1, 92)


# Combine all synthetic data into a single DataFrame and apply inverse transform to bring it back to original scale
synthetic_data = pd.concat(
    [synthetic_data_class_0, synthetic_data_class_1],
    ignore_index=True,
)

# Create corresponding class labels
synthetic_labels = [0] * 56870 + [1] * 92

# Add labels to the synthetic data
synthetic_data["class"] = synthetic_labels

# Save synthetic data as a CSV file
synthetic_data.to_csv(f"../data/interim/generated/CGAN{TRAINING_STEPS}.csv", index=False)

3/3 [==============================] - 0s 1ms/step
